In [2]:
import sys
sys.path.append('..')

In [3]:
from datasets import load_dataset
from util.model import smiles2graph

from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, Batch
from model.gae_gcl import GAEWithPooling
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from itertools import chain
from util.prompt import create_cot_prompt, create_incontext_prompt2
from util.measure import measure
import torch_geometric.utils.smiles as smiles

In [72]:
dataset_name = 'liupf/ChEBI-20-MM'

dataset = load_dataset(dataset_name)
df_train = dataset['train'].to_pandas()
df_valid = dataset['validation'].to_pandas()
df_test = dataset['test'].to_pandas()

def smiles2graph(smiles_str):
    data = smiles.from_smiles(smiles_str)
    data.edge_attr = data.edge_attr.float()
    data.x = data.x.float()
    return Data(x=data.x, edge_index=data.edge_index, edge_attr=data.edge_attr)

In [73]:
df_train.iloc[0]

CID                                                    129626631
SMILES         CCCCC[C@@H]1O[C@@H]1/C=C/C(O)C/C=C\C/C=C\CCCC(...
description    The molecule is an epoxy(hydroxy)icosatrienoat...
polararea                                                   72.9
xlogp                                                        4.6
inchi          InChI=1S/C20H32O4/c1-2-3-9-13-18-19(24-18)16-1...
iupacname      (5Z,8Z,12E)-11-hydroxy-13-[(2R,3S)-3-pentyloxi...
SELFIES        [C][C][C][C][C][C@@H1][O][C@@H1][Ring1][Ring1]...
Name: 0, dtype: object

In [76]:
smiles2graph(df_train.iloc[0]['SMILES']).edge_attr


tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [2., 3., 0.],
        [2., 3., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [2., 2., 0.],
        [2., 2., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [2., 2., 0.],
        [2., 2., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [2., 0., 1.],
        [1

In [75]:
train_graphs = [smiles2graph(smiles) for smiles in df_train['SMILES']]
train_loader = DataLoader(train_graphs, batch_size=16, shuffle=False)

val_graphs = [smiles2graph(smiles) for smiles in df_valid['SMILES']]
val_loader = DataLoader(val_graphs, batch_size=16, shuffle=False)

test_graphs = [smiles2graph(smiles) for smiles in df_test['SMILES']]
test_loader = DataLoader(test_graphs, batch_size=16, shuffle=False)

In [10]:
model = GAEWithPooling(9, 32, 16, 9, train_graphs)  # Replace with your model class
model.load_state_dict(torch.load('/home/ali.lawati/mol-incontext/checkpoints/gcl-gae-100.pt', map_location=torch.device('cpu')))

/tmp/ipykernel_2680543/3312000232.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/ali.lawati/mol-incontext/checkpoints/gcl-gae-10

<All keys matched successfully>

In [11]:
with torch.no_grad():
    train_batch = Batch.from_data_list(train_graphs)#.to(device)
    test_batch  = Batch.from_data_list(test_graphs)#.to(device)
    _, _, train_pool = model(train_batch.x, train_batch.edge_index, train_batch.batch)
    _, _, test_pool = model(test_batch.x, test_batch.edge_index, test_batch.batch)

In [12]:
i = 0
n = 10
a_emb = test_pool[i]

b_idx = torch.topk(F.cosine_similarity(a_emb, train_pool), n).indices.tolist()
#b_idx


# Normalize the vectors to unit length
train_pool_normalized = torch.nn.functional.normalize(train_pool[b_idx], p=2, dim=1)

# Compute the pairwise cosine similarity
similarity_matrix = torch.matmul(train_pool_normalized, train_pool_normalized.T)

#print(similarity_matrix)

In [88]:
data_dict = {
    'prompt1': np.squeeze(prompts1),
    'ref': references
}
df = pd.DataFrame(data_dict)
df.to_csv(f"./input/maxspace--epochs100.csv", index=False)

In [61]:
num_examples = 4
B_ALPHA = 0.3
def get_samples_new(a_emb, df_train, train_pool, num_examples):
    alpha = B_ALPHA
    selected_indices = []
    combined_score = F.cosine_similarity(a_emb, train_pool)
    for _ in range(num_examples):
        top20 = torch.topk(combined_score, 20)[1]
        idx = next((i for i in top20 if i not in selected_indices), None).item()
        selected_indices.append(idx)
        
        b_emb = train_pool[idx]
        similarity_to_selected =  F.cosine_similarity(b_emb, train_pool)
        combined_score -= alpha * similarity_to_selected
        alpha = alpha - .3*(alpha)

    #print(selected_indices)
    sampled_data = df_train.iloc[selected_indices]
    return list(chain.from_iterable(zip(sampled_data['SMILES'], sampled_data['description'])))

train_pool_normalized = train_pool / train_pool.norm(dim=1, keepdim=True)

references, prompts1 = [], []
for i in range(0, len(df_test)):
    prog = df_test.iloc[i]['SMILES']
    p_args1 = get_samples_new(test_pool[i], df_train, train_pool_normalized, num_examples) + [prog]
    prompt1 = create_incontext_prompt2(*p_args1)

    prompts1.append(prompt1)
    references.append(df_test.iloc[i]['description'])

data_dict = {
    'prompt1': np.squeeze(prompts1),
    'ref': references
}
df = pd.DataFrame(data_dict)
df.to_csv(f"../../input/gae-gcl-customloss-{num_examples}-dimAlpha-epochs100-ensure-diff.csv", index=False)

[25083, 17569, 7195, 25303]
[2592, 4896, 31, 19997]
[24121, 1736, 14432, 13057]
[14062, 21511, 19830, 4384]
[19201, 17406, 11870, 93]
[20984, 8326, 1168, 19163]
[5144, 10491, 6571, 7791]
[9841, 9788, 24785, 18752]
[24799, 9722, 4530, 9628]
[15955, 889, 18117, 24565]
[16205, 19086, 10209, 14879]
[15979, 23103, 19297, 6466]
[9546, 24790, 8405, 3939]
[2109, 11683, 1169, 7978]
[20696, 21612, 17964, 2926]
[26104, 8327, 12242, 3502]
[22675, 11148, 8814, 391]
[17516, 11532, 18921, 25314]
[19356, 11305, 21650, 20996]
[1604, 21846, 11358, 16242]
[2310, 8018, 24882, 23918]
[15545, 10121, 23329, 5827]
[15377, 23348, 19427, 26032]
[23436, 12314, 9391, 14947]
[11346, 12750, 18251, 22828]
[13183, 8325, 14557, 5164]
[23522, 4475, 992, 8482]
[20044, 12644, 16066, 3042]
[6248, 13145, 7118, 11389]
[22007, 17034, 22674, 8356]
[826, 7935, 3409, 17809]
[10875, 2461, 20424, 24351]
[11121, 13570, 5051, 1306]
[10887, 23290, 11101, 24600]
[3919, 19345, 14013, 23335]
[8824, 8384, 1647, 10422]
[1415, 20838, 2549

In [29]:

train_pool_normalized = train_pool / train_pool.norm(dim=1, keepdim=True)
print(train_pool_normalized)

tensor([[-0.0230, -0.1014,  0.0717,  ...,  0.2276,  0.3591,  0.0883],
        [ 0.1566, -0.1265,  0.4414,  ..., -0.5524,  0.1127, -0.0509],
        [-0.0129, -0.0075, -0.2315,  ..., -0.6825, -0.1682, -0.0333],
        ...,
        [ 0.1454,  0.0484,  0.2102,  ...,  0.3680,  0.1334, -0.0565],
        [-0.2582,  0.2681,  0.1763,  ...,  0.2221,  0.1939, -0.1374],
        [ 0.0461,  0.0522, -0.0593,  ..., -0.1723, -0.2279, -0.1829]])


## Test Maximal Span

In [15]:
def select_maximal_span(vectors):
    num_vectors, vector_dim = vectors.shape
    selected_indices = []
    selected_vectors = []

    # Track the orthogonal components of remaining vectors
    orthogonal_components = vectors.clone()
    
    for _ in range(num_vectors):
        # Calculate the norms of the orthogonal components
        norms = torch.norm(orthogonal_components, dim=1)
        
        # Select the vector with the maximum norm
        max_index = torch.argmax(norms).item()
        
        # Check if the norm is effectively zero
        if norms[max_index] < 1e-6:  # Tolerance for numerical precision
            break
        
        # Add this vector to the selected set
        selected_indices.append(max_index)
        selected_vector = vectors[max_index]
        selected_vectors.append(selected_vector)
        
        # Orthogonalize remaining vectors against the selected vector
        for i in range(num_vectors):
            projection = torch.dot(orthogonal_components[i], selected_vector) / torch.dot(selected_vector, selected_vector)
            orthogonal_components[i] -= projection * selected_vector
    
    selected_vectors = torch.stack(selected_vectors)
    return selected_vectors, selected_indices

    
# Perform selection to maximize span
for i in range(100):
    a_emb = test_pool[i]
    b_idx = torch.topk(F.cosine_similarity(a_emb, train_pool), n).indices.tolist()
    sampled_data = df_train.iloc[b_idx]
    selected_vectors, selected_indices = select_maximal_span(train_pool[b_idx])
    list(chain.from_iterable(zip(sampled_data['SMILES'], sampled_data['description'])))
    #print("Indices of selected vectors:")
    #print(list(set(selected_indices)))
#print("Selected Vectors that maximize the span:")
#print(selected_vectors)

#eigenvalues, eigenvectors = torch.linalg.eig(train_pool[b_idx])

In [87]:
references, prompts1 = [], []
for i in range(0, len(df_test)):
    prog = df_test.iloc[i]['SMILES']
    a_emb = test_pool[i]
    b_idx = torch.topk(F.cosine_similarity(a_emb, train_pool), n).indices.tolist()
    sampled_data = df_train.iloc[b_idx].reset_index()
    #print(sampled_data['SMILES'])
    #print(b_idx)
    selected_vectors, selected_indices = select_maximal_span(train_pool[b_idx])
    if (selected_indices == None):
        print ('big ooooops')
    selected_indices.append(0)
    #print(selected_indices)
    #print(list(set(selected_indices)), print(sampled_data.iloc[0]['SMILES']))
    sampled_data = sampled_data.iloc[list(set(selected_indices))]
    p_args1 = list(chain.from_iterable(zip(sampled_data['SMILES'], sampled_data['description']))) + [prog]
    prompt1 = create_incontext_prompt2(*p_args1)

    prompts1.append(prompt1)
    references.append(df_test.iloc[i]['description'])